# Feathr Local Spark Quickstart - NYC Taxi Demo
This notebook demonstrates how to use Feathr to train a model on a local Spark cluster. We will use the NYC Taxi dataset to predict the tip amount for a taxi ride. The dataset is available on [Kaggle](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/data).

## 1. Install Feathr and Necessary Dependancies

Install feathr and necessary packages by running `pip install feathr[notebook]` if you haven't installed them already.

In [2]:
%pip install -U --user feathr pandavro scikit-learn

  Using cached feathr-0.9.0-py3-none-any.whl (112 kB)
  Using cached pandavro-1.7.1-py3-none-any.whl
  Using cached scikit_learn-1.2.0-cp38-cp38-macosx_10_9_x86_64.whl (8.9 MB)
  Using cached confluent_kafka-1.9.2-cp38-cp38-macosx_10_9_x86_64.whl (2.8 MB)
  Using cached azure_synapse_spark-0.7.0-py2.py3-none-any.whl (32 kB)
  Using cached pyarrow-9.0.0-cp38-cp38-macosx_10_13_x86_64.whl (24.0 MB)
  Using cached azure_core-1.22.1-py3-none-any.whl (178 kB)
  Using cached redis-4.4.0-py3-none-any.whl (236 kB)
  Using cached deltalake-0.6.4-cp37-abi3-macosx_10_7_x86_64.whl (13.4 MB)
  Using cached pyapacheatlas-0.14.0-py3-none-any.whl (74 kB)
  Using cached loguru-0.6.0-py3-none-any.whl (58 kB)
  Using cached azure_keyvault_secrets-4.6.0-py3-none-any.whl (291 kB)
  Using cached graphlib_backport-1.0.3-py3-none-any.whl (5.1 kB)
  Using cached avro-1.11.1-py2.py3-none-any.whl
  Using cached pyhocon-0.3.59-py3-none-any.whl
  Using cached msrest-0.6.21-py2.py3-none-any.whl (85 kB)
  Using cache

In [286]:
# This step is optional and only required if you want to use the feathr registry
#! az login --use-device-code

## 2. Create Shareable Features with Feathr Feature Definition Configs

In this notebook, we define all the necessary resource key values for authentication. We use the values passed by the databricks widgets at the top of this notebook. Instead of manually entering the values to the widgets, we can also use [Azure Key Vault](https://azure.microsoft.com/en-us/services/key-vault/) to retrieve them.
Please refer to [how-to guide documents for granting key-vault access](https://feathr-ai.github.io/feathr/how-to-guides/azure-deployment-arm.html#3-grant-key-vault-and-synapse-access-to-selected-users-optional) and [Databricks' Azure Key Vault-backed scopes](https://learn.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes) for more details.

In [3]:
import glob
import os
import tempfile
from datetime import datetime, timedelta
from math import sqrt

import pandas as pd
import pandavro as pdx
from feathr import FeathrClient
from feathr import BOOLEAN, FLOAT, INT32, ValueType
from feathr import Feature, DerivedFeature, FeatureAnchor
from feathr import BackfillTime, MaterializationSettings
from feathr import FeatureQuery, ObservationSettings
from feathr import RedisSink
from feathr import INPUT_CONTEXT, HdfsSource
from feathr import WindowAggTransformation
from feathr import TypedKey
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from azure.identity import AzureCliCredential
from azure.keyvault.secrets import SecretClient
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.sql import DataFrame, SparkSession
import pyspark.sql.functions as F

In [4]:
import feathr
print(feathr.__version__)

0.9.0


## Initialize Feathr Client 
We will use the Feathr Local client to train our model. The client will be initialized with the default configuration. The default configuration can be overwritten by environment variables.

In [5]:
os.environ['SPARK_LOCAL_IP'] = "127.0.0.1"

import tempfile
yaml_config = f"""
api_version: 1
project_config:
  project_name: 'local_spark'
  required_environment_variables:
  optional_environment_variables:
    # the environemnt variables are optional, however you will need them if you want to use some of the services:
    - ADLS_ACCOUNT
    - ADLS_KEY
    - WASB_ACCOUNT
    - WASB_KEY
    - S3_ACCESS_KEY
    - S3_SECRET_KEY
    - JDBC_TABLE
    - JDBC_USER
    - JDBC_PASSWORD
    - KAFKA_SASL_JAAS_CONFIG


spark_config:
  # choice for spark runtime. Currently support: azure_synapse, databricks, local
  spark_cluster: 'local'
  spark_result_output_parts: '1'
  local:
    master: 'local[*]'
    feathr_runtime_location:

online_store:
  redis:
    # Redis configs to access Redis cluster
    host: '<redis_host_name>'
    port: 6380
    ssl_enabled: True

feature_registry:
  # The API endpoint of the registry service
  api_endpoint: "https://feathr-sql-registry.azurewebsites.net/api/v1"
"""

tmp = tempfile.NamedTemporaryFile(mode='w', delete=False)
with open(tmp.name, "w") as text_file:
    text_file.write(yaml_config)

### Initialize Feathr Client

In [6]:
client = FeathrClient(tmp.name)

2023-01-07 16:12:35.728 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - secrets__azure_key_vault__name not found in the config file.
2023-01-07 16:12:35.749 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - offline_store__s3__s3_enabled not found in the config file.
2023-01-07 16:12:35.754 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - offline_store__adls__adls_enabled not found in the config file.
2023-01-07 16:12:35.759 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - offline_store__wasb__wasb_enabled not found in the config file.
2023-01-07 16:12:35.763 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - offline_store__jdbc__jdbc_enabled not found in the config file.
2023-01-07 16:12:35.768 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - offline_store__snowflake__s

### Download the sample data
The sample data will be downloaded to your workspace.

In [7]:
import pandas as pd
df_raw = pd.read_csv("https://azurefeathrstorage.blob.core.windows.net/public/sample_data/green_tripdata_2020-04_with_index.csv")
df_raw.to_csv("green_tripdata_2020-04_with_index.csv", index=False)
DATA_FILE_PATH = "./green_tripdata_2020-04_with_index.csv"

<ipython-input-7-ed3176f1b792>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv("https://azurefeathrstorage.blob.core.windows.net/public/sample_data/green_tripdata_2020-04_with_index.csv")


### Defining features with Feathr

In Feathr, a feature is viewed as a function, mapping a key and timestamp to a feature value. For more details, please see [Feathr Feature Definition Guide](https://github.com/feathr-ai/feathr/blob/main/docs/concepts/feature-definition.md).

* The feature key (a.k.a. entity id) identifies the subject of feature, e.g. a user_id or location_id.
* The feature name is the aspect of the entity that the feature is indicating, e.g. the age of the user.
* The feature value is the actual value of that aspect at a particular time, e.g. the value is 30 at year 2022.

Note that, in some cases, a feature could be just a transformation function that has no entity key or timestamp involved, e.g. *the day of week of the request timestamp*.

There are two types of features -- anchored features and derivated features:

* **Anchored features**: Features that are directly extracted from sources. Could be with or without aggregation. 
* **Derived features**: Features that are computed on top of other features.

#### Define anchored features

A feature source is needed for anchored features that describes the raw data in which the feature values are computed from. A source value should be either `INPUT_CONTEXT` (the features that will be extracted from the observation data directly) or `feathr.source.Source` object.

In [8]:
TIMESTAMP_COL = "lpep_dropoff_datetime"
TIMESTAMP_FORMAT = "yyyy-MM-dd HH:mm:ss"

In [9]:
def preprocessing(df: DataFrame) -> DataFrame:
    import pyspark.sql.functions as F
    df = df.withColumn("fare_amount_cents", (F.col("fare_amount") * 100.0).cast("float"))
    return df

batch_source = HdfsSource(
    name="nycTaxiBatchSource",
    path=DATA_FILE_PATH,
    event_timestamp_column=TIMESTAMP_COL,
    preprocessing=preprocessing,
    timestamp_format=TIMESTAMP_FORMAT,
)

In [10]:
# We define f_trip_distance and f_trip_time_duration features separately
# so that we can reuse them later for the derived features.
f_trip_distance = Feature(
    name="f_trip_distance",
    feature_type=FLOAT,
    transform="trip_distance",
)
f_trip_time_duration = Feature(
    name="f_trip_time_duration",
    feature_type=FLOAT,
    transform="cast_float((to_unix_timestamp(lpep_dropoff_datetime) - to_unix_timestamp(lpep_pickup_datetime)) / 60)",
)



features = [
    f_trip_distance,
    f_trip_time_duration,
    Feature(
        name="f_is_long_trip_distance",
        feature_type=BOOLEAN,
        transform="trip_distance > 30.0",
    ),
    Feature(
        name="f_day_of_week",
        feature_type=INT32,
        transform="dayofweek(lpep_dropoff_datetime)",
    ),
    Feature(
        name="f_day_of_month",
        feature_type=INT32,
        transform="dayofmonth(lpep_dropoff_datetime)",
    ),
    Feature(
        name="f_hour_of_day",
        feature_type=INT32,
        transform="hour(lpep_dropoff_datetime)",
    ),
]

# After you have defined features, bring them together to build the anchor to the source.
feature_anchor = FeatureAnchor(
    name="feature_anchor",
    source=INPUT_CONTEXT,  # Pass through source, i.e. observation data.
    features=features,
)

For the features with aggregation, the supported functions are as follows:

| Aggregation Function | Input Type | Description |
| --- | --- | --- |
|SUM, COUNT, MAX, MIN, AVG	|Numeric|Applies the the numerical operation on the numeric inputs. |
|MAX_POOLING, MIN_POOLING, AVG_POOLING	| Numeric Vector | Applies the max/min/avg operation on a per entry bassis for a given a collection of numbers.|
|LATEST| Any |Returns the latest not-null values from within the defined time window |

In [11]:
agg_key = TypedKey(
    key_column="DOLocationID",
    key_column_type=ValueType.INT32,
    description="location id in NYC",
    full_name="nyc_taxi.location_id",
)

agg_window = "90d"

# Anchored features with aggregations
agg_features = [
    Feature(
        name="f_location_avg_fare",
        key=agg_key,
        feature_type=FLOAT,
        transform=WindowAggTransformation(
            agg_expr="fare_amount_cents",
            agg_func="AVG",
            window=agg_window,
        ),
    ),
    Feature(
        name="f_location_max_fare",
        key=agg_key,
        feature_type=FLOAT,
        transform=WindowAggTransformation(
            agg_expr="fare_amount_cents",
            agg_func="MAX",
            window=agg_window,
        ),
    ),
]

agg_feature_anchor = FeatureAnchor(
    name="agg_feature_anchor",
    source=batch_source,  # External data source for feature. Typically a data table.
    features=agg_features,
)

#### Define derived features

We also define a derived feature, `f_trip_distance_rounded`, from the anchored features `f_trip_distance` as follows:

In [12]:
f_trip_time_distance = DerivedFeature(name="f_trip_time_distance",
                                          feature_type=FLOAT,
                                          input_features=[
                                              f_trip_distance, f_trip_time_duration],
                                          transform="f_trip_distance * f_trip_time_duration")

f_trip_time_rounded = DerivedFeature(name="f_trip_time_rounded",
                                         feature_type=INT32,
                                         input_features=[f_trip_time_duration],
                                         transform="f_trip_time_duration % 10")

derived_feature = [f_trip_time_distance, f_trip_time_rounded]

### Build features

Finally, we build the features.

In [13]:
client.build_features(
    anchor_list=[feature_anchor, agg_feature_anchor],
    derived_feature_list=derived_feature,
)

In [14]:
# This cell is optional if you want to use the feathr registry.
#client.register_features()
#client.list_registered_features(client.project_name)

## 3. Create Training Data Using Point-in-Time Correct Feature Join

After the feature producers have defined the features (as described in the Feature Definition part), the feature consumers may want to consume those features. Feature consumers will use observation data to query from different feature tables using Feature Query.

To create a training dataset using Feathr, one needs to provide a feature join configuration file to specify
what features and how these features should be joined to the observation data. 

To learn more on this topic, please refer to [Point-in-time Correctness](https://github.com/linkedin/feathr/blob/main/docs/concepts/point-in-time-join.md)

In [15]:
feature_names = [feature.name for feature in features + agg_features]
feature_names

['f_trip_distance',
 'f_trip_time_duration',
 'f_is_long_trip_distance',
 'f_day_of_week',
 'f_day_of_month',
 'f_hour_of_day',
 'f_location_avg_fare',
 'f_location_max_fare']

In [16]:
now = datetime.now().strftime("%Y%m%d%H%M%S")
output_path = os.path.join("debug", f"test_output_{now}")

offline_features_path = output_path

In [17]:
# Features that we want to request. Can use a subset of features
query = FeatureQuery(
    feature_list=feature_names,
    key=agg_key,
)
settings = ObservationSettings(
    observation_path=DATA_FILE_PATH,
    event_timestamp_column=TIMESTAMP_COL,
    timestamp_format=TIMESTAMP_FORMAT,
)
client.get_offline_features(
    observation_settings=settings,
    feature_query=query,
    output_path=offline_features_path,
)

client.wait_job_to_finish(timeout_sec=5000)

2023-01-07 16:13:55.670 | WARNING  | feathr.spark_provider._localspark_submission:submit_feathr_job:73 - Local Spark Mode only support basic params right now and should be used only for testing purpose.
2023-01-07 16:13:55.671 | INFO     | feathr.spark_provider._localspark_submission:_init_args:202 - Spark job: local_spark_feathr_feature_join_job is running on local spark with master: local[*].
2023-01-07 16:13:55.682 | INFO     | feathr.spark_provider._localspark_submission:submit_feathr_job:116 - Detail job stdout and stderr are in debug/local_spark_feathr_feature_join_job20230107161355/log.
2023-01-07 16:13:55.685 | INFO     | feathr.spark_provider._localspark_submission:submit_feathr_job:126 - Local Spark job submit with pid: 8716.
2023-01-07 16:13:55.687 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:134 - 1 local spark job(s) in this Launcher, only the latest will be monitored.
2023-01-07 16:13:55.688 | INFO     | feathr.spark_provider._localspark_s

debug/local_spark_feathr_feature_join_job20230107161355
['/var/folders/8k/ph3xqvb11k32hjl7r78jhzh00000gn/T/tmpb1slfns4/feathr_pyspark_driver.py']
x>x>x>x>>x>x>xxx

2023-01-07 16:14:57.747 | WARNING  | feathr.spark_provider._localspark_submission:wait_for_completion:146 - Spark job has hang for 15 seconds. latest msg is downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/3.2.0/hadoop-azure-3.2.0.jar ...
.                             Please check debug/local_spark_feathr_feature_join_job20230107161355/log_0.txt
2023-01-07 16:14:57.748 | WARNING  | feathr.spark_provider._localspark_submission:_clean_up:191 - Terminate the spark job due to as clean_up is set to True.
2023-01-07 16:14:57.789 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:184 - Spark job with pid 8716 finished in: 62 seconds                     with returncode 143


In [19]:
dataframe_list = []
vertical_concat_df = None
for file in glob.glob(os.path.join(output_path, '*.avro')):
    dataframe_list.append(pdx.read_avro(file))
    vertical_concat_df = pd.concat(dataframe_list, axis=0)

vertical_concat_df

## 4. Train and Evaluate a Prediction Model

After generating all the features, we train and evaluate a machine learning model to predict the NYC taxi fare prediction. In this example, we use Spark MLlib's [GBTRegressor](https://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-tree-regression).

Note that designing features, training prediction models and evaluating them are an iterative process where the models' performance maybe used to modify the features as a part of the modeling process.

In [20]:
# remove columns
from sklearn.ensemble import GradientBoostingRegressor
final_df = vertical_concat_df
final_df.drop(["lpep_pickup_datetime", "lpep_dropoff_datetime",
              "store_and_fwd_flag"], axis=1, inplace=True, errors='ignore')
final_df.fillna(0, inplace=True)
final_df['fare_amount'] = final_df['fare_amount'].astype("float64")


train_x, test_x, train_y, test_y = train_test_split(final_df.drop(["fare_amount"], axis=1),
                                                    final_df["fare_amount"],
                                                    test_size=0.2,
                                                    random_state=42)


AttributeError: 'NoneType' object has no attribute 'drop'

In [ ]:
model = GradientBoostingRegressor()
model.fit(train_x, train_y)

y_predict = model.predict(test_x)

y_actual = test_y.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))

sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

In [ ]:
mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)


## 5. Materialize Feature Values for Online Scoring

While we computed feature values on-the-fly at request time via Feathr, we can pre-compute the feature values and materialize them to offline or online storages such as Redis.

Note, only the features anchored to offline data source can be materialized.

In [ ]:
materialized_feature_names = [feature.name for feature in agg_features]
materialized_feature_names

['f_location_avg_fare', 'f_location_max_fare']

In [ ]:
REDIS_KEY = None

In [ ]:
if REDIS_KEY:
    FEATURE_TABLE_NAME = "nycTaxiDemoFeature"

    backfill_time = BackfillTime(start=datetime(
        2020, 4, 1), end=datetime(2020, 4, 1), step=timedelta(days=1))
    redisSink = RedisSink(table_name=FEATURE_TABLE_NAME)
    settings = MaterializationSettings(FEATURE_TABLE_NAME + ".job",
                                       sinks=[redisSink],
                                       feature_names=[
                                           "f_location_avg_fare", "f_location_max_fare"],
                                       backfill_time=backfill_time)
    client.materialize_features(settings)

    client.wait_job_to_finish(5000)

2022-10-31 13:20:07.063 | INFO     | feathr.utils._envvariableutil:get_environment_variable:82 - KAFKA_SASL_JAAS_CONFIG is not set in the environment variables.
2022-10-31 13:20:07.063 | INFO     | feathr.utils._envvariableutil:get_environment_variable:82 - REDIS_PASSWORD is not set in the environment variables.
2022-10-31 13:20:07.065 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - monitoring__database__sql__url not found in the config file.
2022-10-31 13:20:07.068 | INFO     | feathr.utils._envvariableutil:get_environment_variable_with_default:51 - monitoring__database__sql__user not found in the config file.
2022-10-31 13:20:07.069 | INFO     | feathr.utils._envvariableutil:get_environment_variable:82 - MONITORING_DATABASE_SQL_PASSWORD is not set in the environment variables.
2022-10-31 13:20:07.069 | WARNING  | feathr.spark_provider._localspark_submission:submit_feathr_job:73 - Local Spark Mode only support basic params right now and should be 

debug/local_spark_feathr_feature_materialization_job20221031132007
['/var/folders/93/3jr6fmn92v1dwn7y6mmfyvl00000gn/T/tmp1m74z05l/feathr_pyspark_driver.py']
x>x>>>>

KeyboardInterrupt: 

In [ ]:
if REDIS_KEY:
    # Note, to get a single key, you may use client.get_online_features instead
    materialized_feature_values = client.multi_get_online_features(
        feature_table=FEATURE_TABLE_NAME,
        keys=["239", "265"],
        feature_names=materialized_feature_names,
    )
    materialized_feature_values

## Clean up the output of the notebook

In [ ]:
import shutil
shutil.rmtree('debug')